In [1]:
from __future__ import print_function
import numpy as np
import random
import sys

In [2]:
text = open('./shakespeare_final.txt', 'rt', encoding='UTF8').read().lower()

In [3]:
characters = sorted(list(set(text)))

In [4]:
print('corpus length:', len(text))
print('total chars:', len(characters))

corpus length: 196788
total chars: 63


In [5]:
char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

In [6]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 65583


In [7]:
# 인덱스를 벡터화된 형태로 변환
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
    y[i, char2indices[next_chars[i]]] = 1

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM,Activation,Dropout
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [9]:
# 모델 구축
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(characters))))

model.add(Dense(len(characters)))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               98304     
_________________________________________________________________
dense_1 (Dense)              (None, 63)                8127      
_________________________________________________________________
activation_1 (Activation)    (None, 63)                0         
Total params: 106,431
Trainable params: 106,431
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# 예측을 인덱스로 변환하는 함수
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [11]:
# 모델 학습 및 평가
for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.7,1.2]:

        print('\n----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]

            generated += pred_char
            sentence = sentence[1:] + pred_char

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\n하나의 조합 완료\n")

----------------------------------------
Iteration 1
Epoch 1/1
21120/65583 [========>.....................] - ETA: 35s - loss: 2.6661

KeyboardInterrupt: 